In [521]:
import numpy as np

In [522]:
a = np.array([[3, 2, 1],
              [1, 2, 3],
              [1, 5, np.nan],
              [np.nan, 8, 9],
              [2, 2, 2]])

In [523]:
def drop_na_naive(a, axis=0):
    if axis == 0:
        return a[~np.isnan(a).any(1), :]
    else:
        return a[:, ~np.isnan(a).any(0).T]

In [524]:
def fill_na(a, value='mean', inplace=False):
    if not inplace:
        a = np.copy(a)
    f = np.nanmean
    if value == 'median':
        f = np.nanmedian
    elif value == 'mode':
        f = lambda col: np.unique(col)[0]
    for i in range(a.shape[1]):
        col_mean = f(a[:, i])
        a[np.where(np.isnan(a[:, i])), i] = col_mean
    return a

In [525]:
class LinearRegression:
    def __init__(self, alpha=0.05, max_iter=100):
        self.alpha = alpha
        self.max_iter = max_iter
        self.weights = None
        self.bias = None
    
    def fit(self, X, y):
        self.log = []
        self._init_weights(X)
        n = X.shape[0]
        for epoch in range(self.max_iter):
            y_pred = X @ self.weights + self.bias
            err = y_pred - y
            try:
                self.weights -= self.alpha * (err.T @ X).T / n
            except ValueError as e:
                print("err.T @ X shape:", (err.T @ X).shape)
                print("weights shape:", self.weights.shape)
                print(e)
                break
            self.bias -= self.alpha * np.mean(err)
                
    def predict(self, X):
        return X @ self.weights + self.bias
    
    def _init_weights(self, X):
        self.weights = np.random.randn(X.shape[1], 1) / 100
        if len(self.weights.shape) == 1:
            self.weights = self.weights.reshape(1, -1)
        self.bias = 1

In [537]:
def standardize(a, with_mean=None, with_std=None):
    m = np.nanmean(a, axis=0) if with_mean is None else with_mean
    s = np.nanstd(a, axis=0) if with_std is None else with_std
    a = (a - m) / s
    return a, m, s

def normalize(a, t_min=0, t_max=1):
    maxes = np.nanmax(a, axis=0)
    mins = np.nanmin(a, axis=0)
    a = (a - mins)/(maxes - mins)
    return a, maxes, mins

In [527]:
def fill_lr(predictors, target):
    """
    Assuming predictors to have all data necessary
    """
    print("target shape:", target.shape)
    not_nan_idx = ~np.isnan(target)
    if len(predictors.shape) == 1:
        predictors = predictors[:, np.newaxis]
    if len(target.shape) == 1:
        target = target[:, np.newaxis]
    print("NOT NAN IDX:", not_nan_idx)
    print("SHAPE:", predictors.squeeze().shape)
    X_train, y_train = predictors[not_nan_idx, :], target[not_nan_idx, :]
    X_train, train_mean, train_std = standardize(X_train)
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    X_test, _, _ = standardize(predictors[~not_nan_idx, :], train_mean, train_std)
    target[~not_nan_idx] = lr.predict(X_test).squeeze()

In [531]:
def fill_lr_all(a):
    full_columns = a[:, np.where(~np.isnan(a).any(0))].squeeze()
    if len(full_columns.shape) == 1:
        full_columns = full_columns[:, np.newaxis]
#     print("shape:", full_columns.shape)
    non_full_columns_idx = np.where(np.isnan(a).any(0))
#     print("non_full_columns_idx:", non_full_columns_idx[0].shape)
    for idx in non_full_columns_idx[0]:
#         print("idx:", idx)
#         print("full_columns shape:", full_columns.shape, "missed column:", a[:, idx].shape)
        fill_lr(full_columns, a[:, idx])
        full_columns = np.hstack([full_columns, a[:, idx][:, np.newaxis]])
    return a

In [538]:
normalize(a)

(array([[1.        , 0.        , 0.        ],
        [0.33199278, 0.        , 0.25      ],
        [0.33199278, 0.5       , 0.55809933],
        [0.        , 1.        , 1.        ],
        [0.66599639, 0.        , 0.125     ]]),
 array([3., 8., 9.]),
 array([0.00602039, 2.        , 1.        ]))